In [ ]:
import os
import glob
import json
import pandas as pd


DATA_DIR = '/kaggle/input/finefs/annotation/annotation'  # adjust as needed


element_rows    = []
component_rows  = []


for json_path in glob.glob(os.path.join(DATA_DIR, '*.json')):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    
    sample_id = os.path.splitext(os.path.basename(json_path))[0]
    
    
    for elem_key, elem in data.get('executed_element', {}).items():
        element_rows.append({
            'sample_id':    sample_id,
            'element_key':  elem_key,
            'element_name': elem.get('element'),
            'goe':          elem.get('goe'),
            'coarse_class': elem.get('coarse_class'),
            'jump_comb':    elem.get('jump_comb'),
            'panel_score':  elem.get('score_of_pannel')
        })
    
   
    for comp_key, comp in data.get('program_component', {}).items():
        component_rows.append({
            'sample_id':   sample_id,
            'component':   comp_key,
            'panel_score': comp.get('score_of_pannel')
        })


df_elements   = pd.DataFrame(element_rows)
df_components = pd.DataFrame(component_rows)


df_elements.to_csv('elements_summary.csv', index=False)
df_components.to_csv('components_summary.csv', index=False)


print("=== Elements DataFrame ===")
print(df_elements.head())

print("\n=== Components DataFrame ===")
print(df_components.head(10))

In [7]:
import numpy as np

npz_path = '/kaggle/input/finefs/skeleton/skeleton/1.npz'  # pick any file
with np.load(npz_path) as archive:
    print("Available keys:", archive.keys())

Available keys: KeysView(NpzFile '/kaggle/input/finefs/skeleton/skeleton/1.npz' with keys: reconstruction)


In [2]:


import torch 


user_npz_dir = '/kaggle/input/finefs/skeleton/skeleton'
user_scaler_save_path_base = 'lstm_feature_scaler_focused' 
user_best_model_path = 'best_lstm_model_state_focused.pth' 
user_best_scaler_path = 'best_lstm_scaler_focused.joblib' 
user_model_save_dir = './lstm_trained_models_focused' 

# Device
user_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Component names (usually fixed)
user_components_list = ['skating_skills', 'transitions', 'performance', 'composition', 'interpretation']
user_output_dim = len(user_components_list) 


user_tuning_configuration_templates = [
    
    {
        'name': 'lstm_lr_batch_wd',
        'params': {
            'MODEL_TYPE': 'LSTM', 
            'SEQ_LEN': 1000, 
            'INCLUDE_VELOCITIES': True, 
            'INCLUDE_ACCELERATIONS': False, 
            'POSITION_ENCODING': 'absolute', 
            'MODEL_POOLING_STRATEGY': 'last',
            'HIDDEN_DIM': 256, 
            'NUM_LAYERS': 2,   
            'LSTM_DROPOUT': 0.3, 
            'FC_DROPOUT': 0.4, 
            'OPTIMIZER_TYPE': 'AdamW', 

            
            'LEARNING_RATE': (1e-4, 5e-4, 1e-3), 
            'BATCH_SIZE': (16, 32),              
            'WEIGHT_DECAY': (0.0, 1e-4, 1e-5),   

            'GRADIENT_CLIP_VALUE': 1.0, 
            'NUM_EPOCHS': 10, 
            'PATIENCE': 5, 
            'MIN_DELTA': 0.0001, 
        }
    }, 

    
    {
        'name': 'lstm_arch_dropout',
        'params': {
            'MODEL_TYPE': 'LSTM',
            'SEQ_LEN': 1000,
            'INCLUDE_VELOCITIES': True,
            'INCLUDE_ACCELERATIONS': False,
            'POSITION_ENCODING': 'absolute',
            'MODEL_POOLING_STRATEGY': 'last',

           
            'HIDDEN_DIM': (128, 256), 
            'NUM_LAYERS': (2, 3),   
            'LSTM_DROPOUT': (0.2, 0.4), 
            'FC_DROPOUT': (0.3, 0.5), 

            'OPTIMIZER_TYPE': 'AdamW',
            'LEARNING_RATE': 1e-4, 
            'BATCH_SIZE': 16, # Fixed
            'WEIGHT_DECAY': 1e-4, # Fixed
            'GRADIENT_CLIP_VALUE': 1.0,
            'NUM_EPOCHS': 10,
            'PATIENCE': 5,
            'MIN_DELTA': 0.0001,
        }
    }, 



print("Configuration parameters and focused LSTM tuning templates defined (~42 runs total).")

Configuration parameters and focused LSTM tuning templates defined (~42 runs total).


In [ ]:


import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import joblib 
import itertools 




NPZ_DIR = user_npz_dir
SCALER_SAVE_PATH_BASE = user_scaler_save_path_base
BEST_MODEL_PATH = user_best_model_path
BEST_SCALER_PATH = user_best_scaler_path
DEVICE = user_device
COMPONENTS_LIST = user_components_list
OUTPUT_DIM = user_output_dim # Should match len(COMPONENTS_LIST)

print(f"Script is using device: {DEVICE}")
print(f"Script is using NPZ_DIR: {NPZ_DIR}")



def expand_configurations(template_configs):
    expanded_configs = []
    for template in template_configs:
        template_name = template.get('name', 'unnamed_template')
        template_params = template.get('params', {})

        
        multi_value_params = {
            k: v for k, v in template_params.items()
            if isinstance(v, (tuple, list))
        }
        single_value_params = {
            k: v for k, v in template_params.items()
            if not isinstance(v, (tuple, list))
        }

        if not multi_value_params:
            expanded_configs.append({'name': template_name, 'params': template_params.copy()}) 
        else:
            param_names = list(multi_value_params.keys())
            param_values = list(multi_value_params.values())

            for i, combination in enumerate(itertools.product(*param_values)):
                new_run_params = single_value_params.copy()
                combination_dict = dict(zip(param_names, combination))
                new_run_params.update(combination_dict)

                # Generate a unique name for this specific run
                param_suffix = "_".join([f"{k}{v}" for k, v in combination_dict.items()])
                param_suffix = param_suffix.replace('.', 'p').replace('-', 'm').replace('e-', 'e').replace('[','').replace(']','').replace('(','').replace(')','').replace(',', '_').replace(' ', '')
                # Ensure name is not too long for filenames
                if len(param_suffix) > 50: param_suffix = param_suffix[:50] + '...'
                new_run_name = f"{template_name}_{param_suffix}_{i+1}"

                expanded_configs.append({'name': new_run_name, 'params': new_run_params})

    return expanded_configs



class SkeletonPCSDataset(Dataset):
    def __init__(self, manifest_df, npz_dir, seq_len, components_list,
                 scaler=None, include_velocities=True):
        self.df = manifest_df
        self.npz_dir = npz_dir
        self.seq_len = seq_len
        self.components_list = components_list
        self.include_velocities = include_velocities
        self.scaler = scaler

    def _load_and_process_features(self, sample_id_str):
        file_path = os.path.join(self.npz_dir, sample_id_str + '.npz')
        try:
            if not os.path.exists(file_path):
                
                 return None

            data = np.load(file_path)['reconstruction']
        except Exception as e:
            
            return None

        T, J, C = data.shape
        if T == 0:
           
            return None

        x_coords = data.reshape(T, J * C).astype(np.float32)

        if self.include_velocities:
            if T > 1:
                velocities = np.diff(x_coords, axis=0)
                velocities = np.vstack([np.zeros((1, J * C), dtype=np.float32), velocities])
            else:
                velocities = np.zeros_like(x_coords, dtype=np.float32)
            x_combined = np.concatenate([x_coords, velocities], axis=1)
        else:
            x_combined = x_coords

        return x_combined

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        sample_id_str = str(row['sample_id']).replace('.npz', '')

        x_processed_unscaled = self._load_and_process_features(sample_id_str)

        if x_processed_unscaled is None:
            base_coord_dim_assumption = 17 * 3
            features_dim = base_coord_dim_assumption * (2 if self.include_velocities else 1)
           
            return torch.zeros(self.seq_len, features_dim), torch.zeros(len(self.components_list))

        current_T, current_features_dim = x_processed_unscaled.shape

        if self.scaler is not None and hasattr(self.scaler, 'mean_') and self.scaler.mean_ is not None:
            try:
                if current_features_dim != self.scaler.mean_.shape[0]:
                     print(f"Warning: Feature dimension mismatch for sample {sample_id_str}. Data has {current_features_dim} features, scaler expects {self.scaler.mean_.shape[0]}. Skipping scaling.")
                     x_scaled = x_processed_unscaled
                else:
                    x_scaled = self.scaler.transform(x_processed_unscaled)
            except Exception as e:
                print(f"Error scaling features for {sample_id_str}: {e}. Using unscaled features.")
                x_scaled = x_processed_unscaled
        else:
            x_scaled = x_processed_unscaled

        if current_T < self.seq_len:
            pad_shape = (self.seq_len - current_T, current_features_dim)
            pad = np.zeros(pad_shape, dtype=np.float32)
            x_final = np.vstack([x_scaled, pad])
        else:
            x_final = x_scaled[:self.seq_len]

        y = row[self.components_list].values.astype(np.float32)
        return torch.from_numpy(x_final), torch.from_numpy(y)


def fit_feature_scaler(manifest_df, npz_dir, include_velocities):
    print("Collecting data to fit the scaler...")
    all_features_list = []
    temp_dataset = SkeletonPCSDataset(manifest_df, npz_dir, 1, ['dummy'],
                                      scaler=None, include_velocities=include_velocities)

    valid_samples_count = 0
    for idx in range(len(manifest_df)):
        features = temp_dataset._load_and_process_features(manifest_df.iloc[idx]['sample_id'].replace('.npz', ''))
        if features is not None and features.shape[0] > 0:
             all_features_list.append(features)
             valid_samples_count += 1

    if not all_features_list:
        print("Warning: No features collected to fit the scaler.")
        return None

    all_features_flat = np.vstack(all_features_list)
    print(f"Fitting scaler on {all_features_flat.shape[0]} samples with {all_features_flat.shape[1]} features each from {valid_samples_count} files.")

    scaler = StandardScaler()
    scaler.fit(all_features_flat)
    return scaler


class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, lstm_dropout, fc_dropout, pooling_strategy='mean'):
        super().__init__()
        self.pooling_strategy = pooling_strategy
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, bidirectional=True,
                            dropout=lstm_dropout if num_layers > 1 else 0)

        fc_input_dim = hidden_dim * 2

        self.fc = nn.Sequential(
            nn.Linear(fc_input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(fc_dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(fc_dropout),
            nn.Linear(hidden_dim // 2, output_dim)
        )


    def forward(self, x):
        outs, (hn, cn) = self.lstm(x)

        if self.pooling_strategy == 'mean':
            pooled = torch.mean(outs, dim=1)
        elif self.pooling_strategy == 'last':
            pooled = torch.cat((hn[-2,:,:], hn[-1,:,:]), dim=1)
        else:
            pooled = torch.mean(outs, dim=1)

        return self.fc(pooled)


def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, patience, min_delta, gradient_clip_value):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        for x_b, y_b in train_loader:
            x_b, y_b = x_b.to(device), y_b.to(device)
            optimizer.zero_grad()
            preds = model(x_b)
            loss = criterion(preds, y_b)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_value)
            optimizer.step()
            running_train_loss += loss.item() * x_b.size(0)
        epoch_train_loss = running_train_loss / len(train_loader.dataset)
        history['train_loss'].append(epoch_train_loss)

        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                val_preds = model(x_val)
                val_loss = criterion(val_preds, y_val)
                running_val_loss += val_loss.item() * x_val.size(0)
        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        history['val_loss'].append(epoch_val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.2e}")
        scheduler.step(epoch_val_loss)

        if epoch_val_loss < best_val_loss - min_delta:
            best_val_loss = epoch_val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict()
            print(f"Validation loss improved to {best_val_loss:.4f}. Saving model state.")
        else:
            epochs_no_improve += 1
            print(f"Validation loss did not improve for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break

    if best_model_state:
        model.load_state_dict(best_model_state)
        print("Loaded best model state based on validation loss.")
    else:
         print("No improvement meeting min_delta. Using model state from last epoch trained.")


    return model, history, best_val_loss



def evaluate_model(model, test_loader, criterion, device, components_list, target_means_train, target_stds_train):
    model.eval()
    preds_list, trues_list = [], []
    test_loss = 0.0

    with torch.no_grad():
        for x_b, y_b_normalized in test_loader:
            x_b, y_b_normalized = x_b.to(device), y_b_normalized.to(device)
            p_normalized = model(x_b)
            loss = criterion(p_normalized, y_b_normalized)
            test_loss += loss.item() * x_b.size(0)
            preds_list.append(p_normalized.cpu().numpy())
            trues_list.append(y_b_normalized.cpu().numpy())

    avg_test_loss = test_loss / len(test_loader.dataset)

    preds_np_normalized = np.vstack(preds_list)
    trues_np_normalized = np.vstack(trues_list)

    preds_denorm = preds_np_normalized * target_stds_train + target_means_train
    trues_denorm = trues_np_normalized * target_stds_train + target_means_train

    mse_per_component = mean_squared_error(trues_denorm, preds_denorm, multioutput='raw_values')
    mae_per_component = mean_absolute_error(trues_denorm, preds_denorm, multioutput='raw_values')
    r2_per_component = r2_score(trues_denorm, preds_denorm, multioutput='raw_values')

    metrics_df = pd.DataFrame({
        'Component': components_list, 'MSE': mse_per_component,
        'MAE': mae_per_component, 'R2': r2_per_component
    })
    return metrics_df, trues_denorm, preds_denorm



def run_single_config(config_entry, data_splits, target_norm_params, fixed_params):

    config_name = config_entry.get('name', f'run_{id(config_entry)}')
    config_params = config_entry.get('params', {})

    print(f"\n--- Running Config: {config_name} ---")
    print("Parameters:", config_params)


  
    seq_len = config_params.get('SEQ_LEN', 1000)
    include_velocities = config_params.get('INCLUDE_VELOCITIES', True)
    model_pooling_strategy = config_params.get('MODEL_POOLING_STRATEGY', 'last')
    hidden_dim = config_params.get('HIDDEN_DIM', 256)
    num_layers = config_params.get('NUM_LAYERS', 2)
    # LSTM dropout is only applied if num_layers > 1
    lstm_dropout = config_params.get('LSTM_DROPOUT', 0.3) if num_layers > 1 else 0.0
    fc_dropout = config_params.get('FC_DROPOUT', 0.4)
    optimizer_type = config_params.get('OPTIMIZER_TYPE', 'AdamW')
    learning_rate = config_params.get('LEARNING_RATE', 1e-4)
    batch_size = config_params.get('BATCH_SIZE', 16)
    num_epochs = config_params.get('NUM_EPOCHS', 150)
    weight_decay = config_params.get('WEIGHT_DECAY', 1e-4)
    gradient_clip_value = config_params.get('GRADIENT_CLIP_VALUE', 1.0)
    patience = config_params.get('PATIENCE', 15)
    min_delta = config_params.get('MIN_DELTA', 0.0001)



    npz_dir = fixed_params['NPZ_DIR']
    device = fixed_params['DEVICE']
    components_list = fixed_params['COMPONENTS_LIST']
    output_dim = fixed_params['OUTPUT_DIM']
    scaler_save_path_base = fixed_params['SCALER_SAVE_PATH_BASE']


 
    scaler = fit_feature_scaler(data_splits['train_df_norm'], npz_dir, include_velocities)
    current_scaler_save_path = f"{scaler_save_path_base}_{config_name}.joblib"
    if scaler is not None:
        try:
            joblib.dump(scaler, current_scaler_save_path)
            print(f"Fitted scaler saved to {current_scaler_save_path}")
        except Exception as e:
            print(f"Warning: Could not save scaler for config {config_name}: {e}")
            current_scaler_save_path = None
    else:
        print(f"Warning: Scaler could not be fitted for config {config_name}. Feature scaling will be skipped for this run.")


    
    train_ds = SkeletonPCSDataset(data_splits['train_df_norm'], npz_dir, seq_len, components_list,
                                  scaler=scaler, include_velocities=include_velocities)
    val_ds   = SkeletonPCSDataset(data_splits['val_df_norm'],  npz_dir, seq_len, components_list,
                                  scaler=scaler, include_velocities=include_velocities)
    test_ds  = SkeletonPCSDataset(data_splits['test_df_norm'], npz_dir, seq_len, components_list,
                                 scaler=scaler, include_velocities=include_velocities)

    if len(train_ds) == 0 or len(val_ds) == 0 or len(test_ds) == 0:
         print(f"Skipping config {config_name}: One or more datasets are empty.")
         return {
             'config_name': config_name,
             'config_params': config_params,
             'status': 'Skipped due to empty dataset',
             'best_val_loss': float('inf'),
             'test_metrics': None,
             'history': None,
             'model_state': None,
             'scaler_path': None
         }

   
    non_zero_samples = 0
    for i in range(min(len(train_ds), 100)):
        try:
            x, _ = train_ds[i]
            if torch.sum(x) != 0:
                non_zero_samples += 1
                break
        except Exception as e:
            print(f"Warning: Error loading sample {i} for zero check: {e}")
            continue

    if non_zero_samples == 0 and len(train_ds) > 0:
         print(f"Warning: First {min(len(train_ds), 100)} samples of training dataset for config {config_name} appear to be all zeros after loading/processing. Check NPZ files and path: {npz_dir}")


    actual_batch_size = min(batch_size, len(train_ds))
    if actual_batch_size == 0:
        print(f"Skipping config {config_name}: Effective batch size is 0.")
        return {
             'config_name': config_name, 'config_params': config_params,
             'status': 'Skipped due to zero effective batch size',
             'best_val_loss': float('inf'), 'test_metrics': None,
             'history': None, 'model_state': None, 'scaler_path': None
         }

    train_loader = DataLoader(train_ds, batch_size=actual_batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
    val_loader   = DataLoader(val_ds,   batch_size=min(batch_size, len(val_ds) or 1), shuffle=False, num_workers=2, pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=min(batch_size, len(test_ds) or 1), shuffle=False, num_workers=2, pin_memory=True)


    
    input_dim = None
    try:
        for i in range(len(train_ds)):
             sample_x_raw, _ = train_ds[i]
             if torch.sum(sample_x_raw) != 0:
                 input_dim = sample_x_raw.shape[1]
                 break
        if input_dim is None:
             print(f"CRITICAL ERROR: Could not determine input_dim from training dataset for config {config_name}. All samples seem to have failed loading/processing or are zeros.")
             return {
                'config_name': config_name, 'config_params': config_params,
                'status': 'Skipped due to input_dim determination failure',
                'best_val_loss': float('inf'),
                'test_metrics': None, 'history': None, 'model_state': None, 'scaler_path': None
             }
        print(f"Determined input_dim for config {config_name}: {input_dim}")

    except Exception as e:
        print(f"CRITICAL ERROR: Could not determine input_dim for config {config_name}: {e}. Skipping config.")
        return {
            'config_name': config_name, 'config_params': config_params,
            'status': f'Skipped due to input_dim determination error: {e}',
            'best_val_loss': float('inf'),
            'test_metrics': None, 'history': None, 'model_state': None, 'scaler_path': None
        }


    model = LSTMRegressor(input_dim, hidden_dim, num_layers, output_dim,
                          lstm_dropout, fc_dropout, model_pooling_strategy).to(device)
    criterion = nn.SmoothL1Loss()

    if optimizer_type.lower() == 'adamw':
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    current_scheduler_patience = max(1, patience // 2)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=current_scheduler_patience, verbose=True, min_lr=1e-7)

    
    print(f"Starting training for config {config_name}...")
    try:
        trained_model, history, best_val_loss_achieved = train_model(
            model, train_loader, val_loader, criterion, optimizer, scheduler,
            num_epochs, device, patience, min_delta, gradient_clip_value
        )
        status = 'Completed'
        model_state_dict = trained_model.state_dict()

        plt.figure(figsize=(10, 5))
        plt.plot(history['train_loss'], label='Training Loss')
        plt.plot(history['val_loss'], label='Validation Loss')
        plt.title(f"Config '{config_name}' Training and Validation Loss")
        plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
        plt.savefig(f"loss_plot_{config_name}.png")
        plt.close()

    except Exception as e:
        print(f"Error during training for config {config_name}: {e}")
        status = f'Training failed: {e}'
        best_val_loss_achieved = float('inf')
        history = None
        model_state_dict = None

    
    test_metrics = None
    trues_denorm, preds_denorm = None, None
    if status == 'Completed':
        print(f"Evaluating config {config_name} on the test set...")
        try:
            test_metrics, trues_denorm, preds_denorm = evaluate_model(
                trained_model, test_loader, criterion, device,
                components_list, target_norm_params['target_means_train'], target_norm_params['target_stds_train']
            )
            print(f"\nTest Metrics for Config '{config_name}':")
            print(test_metrics)

            for i, comp in enumerate(components_list):
                plt.figure(figsize=(8, 8))
                plt.scatter(trues_denorm[:, i], preds_denorm[:, i], alpha=0.6, edgecolors='w', linewidth=0.5)
                min_val = min(trues_denorm[:, i].min(), preds_denorm[:, i].min()) - 0.5
                max_val = max(trues_denorm[:, i].max(), preds_denorm[:, i].max()) + 0.5
                plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2, label='Ideal Fit (y=x)')
                plt.xlabel(f"True {comp}"); plt.ylabel(f"Predicted {comp}")
                r2_val = test_metrics.loc[test_metrics['Component'] == comp, 'R2'].values[0] if not test_metrics.empty else float('nan')
                plt.title(f"{comp}: True vs Predicted (Test Set, Config '{config_name}')\nR2: {r2_val:.3f}")
                plt.legend(); plt.grid(True)
                plt.savefig(f"{comp}_true_vs_predicted_plot_{config_name}.png");
                plt.close()

        except Exception as e:
             print(f"Error during evaluation for config {config_name}: {e}")
             status = f'Evaluation failed: {e}'
             test_metrics = None

    return {
        'config_name': config_name,
        'config_params': config_params,
        'status': status,
        'best_val_loss': best_val_loss_achieved,
        'test_metrics': test_metrics.to_dict('records') if test_metrics is not None else None,
        'history': history,
        'model_state': model_state_dict,
        'scaler_path': current_scaler_save_path,
    }



if __name__ == '__main__':
    print("Starting hyperparameter tuning script...")

    
    tuning_configuration_templates = user_tuning_configuration_templates
    print(f"Accessing tuning configurations from 'user_tuning_configuration_templates'.")

    
    print("Expanding configuration templates...")
    expanded_tuning_configurations = expand_configurations(tuning_configuration_templates)
    print(f"Generated {len(expanded_tuning_configurations)} individual configurations to run.")

    if not expanded_tuning_configurations:
        print("No configurations generated from templates. Exiting.")
        exit()


    
    
    if 'df_components' not in locals():
       print("INFO: 'df_components' not found. Creating a dummy DataFrame for demonstration.")
       num_unique_samples_demo = 150
       sample_ids_repeated_demo = [f'sample_{i:03d}' for i in range(num_unique_samples_demo) for _ in range(len(COMPONENTS_LIST))]
       components_repeated_demo = COMPONENTS_LIST * num_unique_samples_demo
       panel_scores_demo = np.random.rand(num_unique_samples_demo * len(COMPONENTS_LIST)) * 7.5 + 2.5
       df_components = pd.DataFrame({
           'sample_id': sample_ids_repeated_demo, 'component': components_repeated_demo,
           'panel_score': panel_scores_demo
       })
       print(f"Created dummy df_components with {len(df_components)} rows.")

       if NPZ_DIR == '/kaggle/input/finefs/skeleton/skeleton':
           print("INFO: NPZ_DIR is the placeholder. Creating dummy NPZ files.")
           dummy_npz_dir = "dummy_npz_data_tuning_v5" # Use a unique folder name
           os.makedirs(dummy_npz_dir, exist_ok=True)
           NPZ_DIR = dummy_npz_dir # Update NPZ_DIR for dummy data
           print(f"Updated NPZ_DIR to '{NPZ_DIR}'. Creating dummy NPZ files...")
           for i in range(num_unique_samples_demo):
               dummy_skeleton_data = np.random.rand(np.random.randint(800,1201), 17, 3)
               np.savez_compressed(os.path.join(NPZ_DIR, f'sample_{i:03d}.npz'), reconstruction=dummy_skeleton_data)
           print(f"Created {num_unique_samples_demo} dummy NPZ files in '{NPZ_DIR}'.")


    
    pcs_df_unnormalized = (
        df_components.loc[df_components['component'].isin(COMPONENTS_LIST)]
        .pivot(index='sample_id', columns='component', values='panel_score')
        .reset_index().rename_axis(None, axis=1)
    )
    pcs_df_unnormalized = pcs_df_unnormalized.dropna(subset=COMPONENTS_LIST)
    if pcs_df_unnormalized.empty:
        raise ValueError("pcs_df_unnormalized is empty after filtering and dropping NaNs. Check df_components and COMPONENTS_LIST.")
    print(f"Created unnormalized PCS DataFrame with {len(pcs_df_unnormalized)} samples.")

    
    train_val_df, test_df_unnorm = train_test_split(pcs_df_unnormalized, test_size=0.2, random_state=42)
    train_df_unnorm, val_df_unnorm = train_test_split(train_val_df, test_size=0.15, random_state=42)

    
    target_means_train = train_df_unnorm[COMPONENTS_LIST].mean().values
    target_stds_train = train_df_unnorm[COMPONENTS_LIST].std().values
    target_stds_train[target_stds_train == 0] = 1.0

    train_df_norm, val_df_norm, test_df_norm = train_df_unnorm.copy(), val_df_unnorm.copy(), test_df_unnorm.copy()
    print(f"Data splits (fixed for all runs): Train={len(train_df_norm)}, Val={len(val_df_norm)}, Test={len(test_df_norm)}")
    for i, col in enumerate(COMPONENTS_LIST):
        train_df_norm[col] = (train_df_unnorm[col] - target_means_train[i]) / target_stds_train[i]
        val_df_norm[col] = (val_df_unnorm[col] - target_means_train[i]) / target_stds_train[i]
        test_df_norm[col] = (test_df_unnorm[col] - target_means_train[i]) / target_stds_train[i]

    TARGET_NORM_PARAMS = {
        'target_means_train': target_means_train,
        'target_stds_train': target_stds_train
    }

    DATA_SPLITS = {
        'train_df_norm': train_df_norm,
        'val_df_norm': val_df_norm,
        'test_df_norm': test_df_norm,
    }

    FIXED_PARAMS = {
        'NPZ_DIR': NPZ_DIR,
        'SCALER_SAVE_PATH_BASE': SCALER_SAVE_PATH_BASE,
        'DEVICE': DEVICE,
        'COMPONENTS_LIST': COMPONENTS_LIST,
        'OUTPUT_DIM': OUTPUT_DIM,
        'BEST_MODEL_PATH': BEST_MODEL_PATH, 
        'BEST_SCALER_PATH': BEST_SCALER_PATH,
    }


    
    all_run_results = []
    best_overall_val_loss = float('inf')
    best_run_result = None

    for config_entry in expanded_tuning_configurations:
        run_result = run_single_config(
            config_entry,
            DATA_SPLITS,
            TARGET_NORM_PARAMS,
            FIXED_PARAMS
        )
        all_run_results.append(run_result)

        if run_result['status'] == 'Completed' and run_result['best_val_loss'] < best_overall_val_loss:
            best_overall_val_loss = run_result['best_val_loss']
            best_run_result = run_result
            print(f"Config '{run_result['config_name']}' is the new best model with validation loss: {best_overall_val_loss:.4f}")


    
    print("\n--- Hyperparameter Tuning Results Summary ---")

    if not all_run_results:
         print("No configurations were attempted.")
    else:
        summary_data = []
        for res in all_run_results:
            summary_row = {'Config': res['config_name'], 'Status': res['status']}
            if res['status'] == 'Completed':
                summary_row['Best Val Loss'] = f"{res['best_val_loss']:.4f}"
                if res['test_metrics']:
                    avg_r2 = np.mean([m['R2'] for m in res['test_metrics']]) if res['test_metrics'] else float('nan')
                    summary_row['Avg Test R2'] = f"{avg_r2:.4f}" if not np.isnan(avg_r2) else 'N/A'
                else:
                    summary_row['Avg Test R2'] = 'N/A'
            else:
                summary_row['Best Val Loss'] = 'N/A'
                summary_row['Avg Test R2'] = 'N/A'

            params = res.get('config_params', {})
            
           
            summary_row['SeqLen'] = params.get('SEQ_LEN', 'N/A')
            summary_row['HiddenDim'] = params.get('HIDDEN_DIM', 'N/A')
            summary_row['Layers'] = params.get('NUM_LAYERS', 'N/A')
            summary_row['LR'] = params.get('LEARNING_RATE', 'N/A')
            summary_row['BatchSize'] = params.get('BATCH_SIZE', 'N/A')
            summary_row['Patience'] = params.get('PATIENCE', 'N/A')
            summary_row['FCDropout'] = params.get('FC_DROPOUT', 'N/A')

            summary_data.append(summary_row)

        if summary_data:
            summary_df = pd.DataFrame(summary_data)
            print(summary_df.to_string())


        if best_run_result:
            print(f"\n--- Overall Best Model: Config '{best_run_result['config_name']}' ---")
            print("Parameters:")
            for key, value in best_run_result.get('config_params', {}).items():
                print(f"  {key}: {value}")

            print(f"Achieved Best Validation Loss during training: {best_run_result['best_val_loss']:.4f}")

            if best_run_result['test_metrics']:
                print("\nTest Set Metrics for the Best Model:")
                best_metrics_df = pd.DataFrame(best_run_result['test_metrics'])
                print(best_metrics_df)
            else:
                 print("\nTest set metrics not available for the best model (evaluation failed).")

            if best_run_result['model_state']:
                try:
                    torch.save(best_run_result['model_state'], FIXED_PARAMS['BEST_MODEL_PATH'])
                    print(f"\nState dictionary of the best model saved to '{FIXED_PARAMS['BEST_MODEL_PATH']}'")
                except Exception as e:
                    print(f"Error saving best model state_dict to '{FIXED_PARAMS['BEST_MODEL_PATH']}': {e}")
            else:
                 print("\nBest model state dictionary is not available (training failed).")


            if best_run_result['scaler_path'] and os.path.exists(best_run_result['scaler_path']):
                 try:
                     best_scaler_dest_path = FIXED_PARAMS['BEST_SCALER_PATH']
                     os.makedirs(os.path.dirname(best_scaler_dest_path) or '.', exist_ok=True)
                     if os.path.abspath(best_run_result['scaler_path']) != os.path.abspath(best_scaler_dest_path):
                        import shutil
                        shutil.copyfile(best_run_result['scaler_path'], best_scaler_dest_path)
                        print(f"Scaler for the best model saved to '{best_scaler_dest_path}'")
                     else:
                        print(f"Scaler for the best model is already at the desired path '{best_scaler_dest_path}'")

                 except Exception as e:
                     print(f"Error saving best model's scaler to '{best_scaler_dest_path}': {e}")
            else:
                 print("\nWarning: Scaler for the best model was not found or not saved.")

        else:
            print("\nNo configuration completed successfully to determine a best model.")

    print("\nScript finished.")

Script is using device: cuda
Script is using NPZ_DIR: /kaggle/input/finefs/skeleton/skeleton
Starting hyperparameter tuning script...
Accessing tuning configurations from 'user_tuning_configuration_templates'.
Expanding configuration templates...
Generated 34 individual configurations to run.
Created unnormalized PCS DataFrame with 1167 samples.
Data splits (fixed for all runs): Train=793, Val=140, Test=234

--- Running Config: lstm_lr_batch_wd_LEARNING_RATE0p0001_BATCH_SIZE16_WEIGHT_DECAY0p0_1 ---
Parameters: {'MODEL_TYPE': 'LSTM', 'SEQ_LEN': 1000, 'INCLUDE_VELOCITIES': True, 'INCLUDE_ACCELERATIONS': False, 'POSITION_ENCODING': 'absolute', 'MODEL_POOLING_STRATEGY': 'last', 'HIDDEN_DIM': 256, 'NUM_LAYERS': 2, 'LSTM_DROPOUT': 0.3, 'FC_DROPOUT': 0.4, 'OPTIMIZER_TYPE': 'AdamW', 'GRADIENT_CLIP_VALUE': 1.0, 'NUM_EPOCHS': 10, 'PATIENCE': 5, 'MIN_DELTA': 0.0001, 'LEARNING_RATE': 0.0001, 'BATCH_SIZE': 16, 'WEIGHT_DECAY': 0.0}
Fitting scaler on 4017543 samples with 102 features each from 793 fi

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10 - Train Loss: 0.4327, Val Loss: 0.4268, LR: 1.00e-04
Validation loss improved to 0.4268. Saving model state.
Epoch 2/10 - Train Loss: 0.4320, Val Loss: 0.4274, LR: 1.00e-04
Validation loss did not improve for 1 epoch(s).
Epoch 3/10 - Train Loss: 0.4294, Val Loss: 0.4303, LR: 1.00e-04
Validation loss did not improve for 2 epoch(s).
Epoch 4/10 - Train Loss: 0.4242, Val Loss: 0.4357, LR: 1.00e-04
Validation loss did not improve for 3 epoch(s).
Epoch 5/10 - Train Loss: 0.4175, Val Loss: 0.4364, LR: 1.00e-05
Validation loss did not improve for 4 epoch(s).
Epoch 6/10 - Train Loss: 0.4174, Val Loss: 0.4375, LR: 1.00e-05
Validation loss did not improve for 5 epoch(s).
Early stopping triggered after 6 epochs.
Loaded best model state based on validation loss.
Evaluating config lstm_lr_batch_wd_LEARNING_RATE0p0001_BATCH_SIZE16_WEIGHT_DECAY0p0_1 on the test set...

Test Metrics for Config 'lstm_lr_batch_wd_LEARNING_RATE0p0001_BATCH_SIZE16_WEIGHT_DECAY0p0_1':
        Component       MSE 

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10 - Train Loss: 0.4331, Val Loss: 0.4265, LR: 1.00e-04
Validation loss improved to 0.4265. Saving model state.
Epoch 2/10 - Train Loss: 0.4322, Val Loss: 0.4271, LR: 1.00e-04
Validation loss did not improve for 1 epoch(s).
Epoch 3/10 - Train Loss: 0.4288, Val Loss: 0.4322, LR: 1.00e-04
Validation loss did not improve for 2 epoch(s).
Epoch 4/10 - Train Loss: 0.4202, Val Loss: 0.4436, LR: 1.00e-04
Validation loss did not improve for 3 epoch(s).
Epoch 5/10 - Train Loss: 0.4153, Val Loss: 0.4424, LR: 1.00e-05
Validation loss did not improve for 4 epoch(s).
Epoch 6/10 - Train Loss: 0.4124, Val Loss: 0.4426, LR: 1.00e-05
Validation loss did not improve for 5 epoch(s).
Early stopping triggered after 6 epochs.
Loaded best model state based on validation loss.
Evaluating config lstm_lr_batch_wd_LEARNING_RATE0p0001_BATCH_SIZE16_WEIGHT_DECAY0p000..._2 on the test set...

Test Metrics for Config 'lstm_lr_batch_wd_LEARNING_RATE0p0001_BATCH_SIZE16_WEIGHT_DECAY0p000..._2':
        Component 

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10 - Train Loss: 0.4348, Val Loss: 0.4291, LR: 1.00e-04
Validation loss improved to 0.4291. Saving model state.
Epoch 2/10 - Train Loss: 0.4330, Val Loss: 0.4280, LR: 1.00e-04
Validation loss improved to 0.4280. Saving model state.
Epoch 3/10 - Train Loss: 0.4308, Val Loss: 0.4299, LR: 1.00e-04
Validation loss did not improve for 1 epoch(s).
Epoch 4/10 - Train Loss: 0.4252, Val Loss: 0.4316, LR: 1.00e-04
Validation loss did not improve for 2 epoch(s).
Epoch 5/10 - Train Loss: 0.4220, Val Loss: 0.4464, LR: 1.00e-04
Validation loss did not improve for 3 epoch(s).
Epoch 6/10 - Train Loss: 0.4149, Val Loss: 0.4440, LR: 1.00e-05
Validation loss did not improve for 4 epoch(s).
Epoch 7/10 - Train Loss: 0.4153, Val Loss: 0.4442, LR: 1.00e-05
Validation loss did not improve for 5 epoch(s).
Early stopping triggered after 7 epochs.
Loaded best model state based on validation loss.
Evaluating config lstm_lr_batch_wd_LEARNING_RATE0p0001_BATCH_SIZE16_WEIGHT_DECAY1em05_3 on the test set...

T

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/10 - Train Loss: 0.4234, Val Loss: 0.4296, LR: 1.00e-04
Validation loss improved to 0.4296. Saving model state.
Epoch 2/10 - Train Loss: 0.4253, Val Loss: 0.4294, LR: 1.00e-04
Validation loss improved to 0.4294. Saving model state.
Epoch 3/10 - Train Loss: 0.4239, Val Loss: 0.4296, LR: 1.00e-04
Validation loss did not improve for 1 epoch(s).
Epoch 4/10 - Train Loss: 0.4182, Val Loss: 0.4318, LR: 1.00e-04
Validation loss did not improve for 2 epoch(s).
Epoch 5/10 - Train Loss: 0.4103, Val Loss: 0.4404, LR: 1.00e-04
Validation loss did not improve for 3 epoch(s).


In [ ]:

class SkeletonPCSDataset(Dataset):
    def __init__(self, manifest_df, npz_dir, seq_len=6000):
        self.df = manifest_df
        self.npz_dir = npz_dir
        self.seq_len = seq_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        data = np.load(os.path.join(self.npz_dir, row['sample_id']+'.npz'))['reconstruction']
        T, J, C = data.shape
        x = data.reshape(T, J*C).astype(np.float32)
        if T < self.seq_len:
            pad = np.zeros((self.seq_len - T, J*C), dtype=np.float32)
            x = np.vstack([x, pad])
        else:
            x = x[:self.seq_len]
        y = np.array([
            row['skating_skills'], row['transitions'],
            row['performance'], row['composition'],
            row['interpretation']
        ], dtype=np.float32)
        return torch.from_numpy(x), torch.from_numpy(y)


train_df, test_df = train_test_split(pcs_df, test_size=0.2, random_state=42)
train_ds = SkeletonPCSDataset(train_df, NPZ_DIR, seq_len=1000)
test_ds  = SkeletonPCSDataset(test_df,  NPZ_DIR, seq_len=1000)


train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=16, shuffle=False, num_workers=4, pin_memory=True)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sample_x, _ = train_ds[0]
input_dim = sample_x.shape[1]


class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_layers=3, output_dim=5, dropout=0.5):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, output_dim)
        )
    def forward(self, x):
        outs, _ = self.lstm(x)
        pooled = torch.mean(outs, dim=1)
        return self.fc(pooled)

        
        
model = LSTMRegressor(input_dim, hidden_dim=128, num_layers=2, output_dim=5).to(device)

criterion = torch.nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


for epoch in range(20):
    model.train()
    for x_b, y_b in train_loader:
        x_b, y_b = x_b.to(device), y_b.to(device)
        optimizer.zero_grad()
        loss = criterion(model(x_b), y_b)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} complete")


model.eval()
preds, trues = [], []
with torch.no_grad():
    for x_b, y_b in test_loader:
        x_b = x_b.to(device)
        p = model(x_b).cpu().numpy()
        preds.append(p)
        trues.append(y_b.numpy())
preds = np.vstack(preds)
target = np.vstack(trues)


preds = preds * target_stds.values + target_means.values
target = target * target_stds.values + target_means.values


components = ['skating_skills','transitions','performance','composition','interpretation']
mse = mean_squared_error(target, preds, multioutput='raw_values')
mae = mean_absolute_error(target, preds, multioutput='raw_values')
metrics = pd.DataFrame({'Component': components, 'MSE': mse, 'MAE': mae})
print(metrics)


for i, comp in enumerate(components):
    plt.figure()
    plt.scatter(target[:, i], preds[:, i], alpha=0.5)
    plt.xlabel(f"True {comp}")
    plt.ylabel(f"Predicted {comp}")
    plt.title(f"{comp}: True vs Predicted")
    plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt


# Paths and Data
NPZ_DIR = 'path/to/your/npz_files' 
MANIFEST_FILE = 'path/to/your/manifest.csv' 


SEQ_LEN = 1000         
HIDDEN_DIM = 256       
NUM_LAYERS = 3         
DROPOUT = 0.5          
OUTPUT_DIM = 5         


LEARNING_RATE = 1e-4   
BATCH_SIZE = 16        
NUM_EPOCHS = 100       
WEIGHT_DECAY = 1e-5    
GRADIENT_CLIP_VALUE = 1.0 

# Early Stopping
PATIENCE = 10          
MIN_DELTA = 0.0001     


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")


class SkeletonPCSDataset(Dataset):
    def __init__(self, manifest_df, npz_dir, seq_len, include_velocities=True):
        self.df = manifest_df
        self.npz_dir = npz_dir
        self.seq_len = seq_len
        self.include_velocities = include_velocities

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Ensure sample_id is a string and doesn't already have .npz
        sample_id_str = str(row['sample_id']).replace('.npz', '')
        file_path = os.path.join(self.npz_dir, sample_id_str + '.npz')

        try:
            data = np.load(file_path)['reconstruction'] 
        except FileNotFoundError:
            print(f"Error: File not found {file_path}")
            
            dummy_features = 17*3 
            if self.include_velocities:
                dummy_features *= 2
            return torch.zeros(self.seq_len, dummy_features), torch.zeros(OUTPUT_DIM)
        except KeyError:
            print(f"Error: 'reconstruction' key not found in {file_path}")
            dummy_features = 17*3
            if self.include_velocities:
                dummy_features *= 2
            return torch.zeros(self.seq_len, dummy_features), torch.zeros(OUTPUT_DIM)


        T, J, C = data.shape 
        x_coords = data.reshape(T, J * C).astype(np.float32)

        if self.include_velocities:
            if T > 1:
                velocities = np.diff(x_coords, axis=0)
                
                velocities = np.vstack([np.zeros((1, J * C), dtype=np.float32), velocities])
            else: 
                velocities = np.zeros_like(x_coords, dtype=np.float32)
            
            x_combined = np.concatenate([x_coords, velocities], axis=1)
        else:
            x_combined = x_coords

        current_T, current_features_dim = x_combined.shape

        
        if current_T < self.seq_len:
            pad = np.zeros((self.seq_len - current_T, current_features_dim), dtype=np.float32)
            x_processed = np.vstack([x_combined, pad])
        else:
            x_processed = x_combined[:self.seq_len]

        y = np.array([
            row['skating_skills'], row['transitions'],
            row['performance'], row['composition'],
            row['interpretation']
        ], dtype=np.float32)

        return torch.from_numpy(x_processed), torch.from_numpy(y)


class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                              batch_first=True, bidirectional=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim), # *2 for bidirectional
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, output_dim)
        )

    def forward(self, x):
        
        outs, _ = self.lstm(x)
       
        
       
        pooled = torch.mean(outs, dim=1)
       
        
        return self.fc(pooled)


def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, patience, min_delta, gradient_clip_value):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        for x_b, y_b in train_loader:
            x_b, y_b = x_b.to(device), y_b.to(device)
            
            optimizer.zero_grad()
            preds = model(x_b)
            loss = criterion(preds, y_b)
            loss.backward()
            
            # Gradient Clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_value)
            
            optimizer.step()
            running_train_loss += loss.item() * x_b.size(0)

        epoch_train_loss = running_train_loss / len(train_loader.dataset)
        history['train_loss'].append(epoch_train_loss)

        # Validation phase
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                val_preds = model(x_val)
                val_loss = criterion(val_preds, y_val)
                running_val_loss += val_loss.item() * x_val.size(0)
        
        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        history['val_loss'].append(epoch_val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_train_loss:.4f}, Val Loss: {epoch_val_loss:.4f}")

       
        scheduler.step(epoch_val_loss)

        
        if epoch_val_loss < best_val_loss - min_delta:
            best_val_loss = epoch_val_loss
            epochs_no_improve = 0
            best_model_state = model.state_dict() 
            print(f"Validation loss improved. Saving model state.")
        else:
            epochs_no_improve += 1
            print(f"Validation loss did not improve for {epochs_no_improve} epoch(s).")

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            if best_model_state:
                model.load_state_dict(best_model_state) 
            break
            
    if best_model_state and epochs_no_improve < patience : # if training finished before early stopping
        print("Training finished. Restoring best model state based on validation loss.")
        model.load_state_dict(best_model_state)

    return model, history


def evaluate_model(model, test_loader, criterion, device, target_means, target_stds, components):
    model.eval()
    preds_list, trues_list = [], []
    test_loss = 0.0

    with torch.no_grad():
        for x_b, y_b in test_loader:
            x_b, y_b_cpu = x_b.to(device), y_b 
            
            p = model(x_b)
            loss = criterion(p, y_b_cpu.to(device)) 
            test_loss += loss.item() * x_b.size(0)
            
            preds_list.append(p.cpu().numpy())
            trues_list.append(y_b_cpu.numpy())

    avg_test_loss = test_loss / len(test_loader.dataset)
    print(f"Average Test Loss: {avg_test_loss:.4f}")

    preds_np = np.vstack(preds_list)
    trues_np = np.vstack(trues_list)

    
    if isinstance(target_means, pd.Series): target_means = target_means.values
    if isinstance(target_stds, pd.Series): target_stds = target_stds.values
    
    preds_denorm = preds_np * target_stds + target_means
    trues_denorm = trues_np * target_stds + target_means
    
    mse_per_component = mean_squared_error(trues_denorm, preds_denorm, multioutput='raw_values')
    mae_per_component = mean_absolute_error(trues_denorm, preds_denorm, multioutput='raw_values')
    r2_per_component = r2_score(trues_denorm, preds_denorm, multioutput='raw_values')
    
    metrics_df = pd.DataFrame({
        'Component': components,
        'MSE': mse_per_component,
        'MAE': mae_per_component,
        'R2': r2_per_component
    })
    
    return metrics_df, trues_denorm, preds_denorm


if __name__ == '__main__':
    
    try:
        pcs_df = pd.read_csv(MANIFEST_FILE)
        # Example: Assuming 'sample_id' column exists and other score columns
        if not all(col in pcs_df.columns for col in ['sample_id', 'skating_skills','transitions','performance','composition','interpretation']):
            raise ValueError("Manifest CSV missing required columns.")
    except FileNotFoundError:
        print(f"Error: Manifest file '{MANIFEST_FILE}' not found. Using dummy data for demonstration.")
        # Create a dummy pcs_df for demonstration if the file is not found
        num_samples = 100
        dummy_data = {
            'sample_id': [f'sample_{i:03d}' for i in range(num_samples)],
            'skating_skills': np.random.rand(num_samples) * 10,
            'transitions': np.random.rand(num_samples) * 10,
            'performance': np.random.rand(num_samples) * 10,
            'composition': np.random.rand(num_samples) * 10,
            'interpretation': np.random.rand(num_samples) * 10
        }
        pcs_df = pd.DataFrame(dummy_data)
        
        
        if NPZ_DIR == 'path/to/your/npz_files':
            print("NPZ_DIR is a placeholder. Creating dummy NPZ files for demonstration.")
            os.makedirs("dummy_npz_data", exist_ok=True)
            NPZ_DIR = "dummy_npz_data"
            for i in range(num_samples):
                # (Time, Joints, Coords) e.g., (random_time, 17 joints, 3 coords)
                # Time steps between 500 and 1500 for variety
                dummy_skeleton_data = np.random.rand(np.random.randint(500,1501), 17, 3)
                np.savez_compressed(os.path.join(NPZ_DIR, f'sample_{i:03d}.npz'), reconstruction=dummy_skeleton_data)
            print(f"Created dummy NPZ files in '{NPZ_DIR}'.")


    components = ['skating_skills','transitions','performance','composition','interpretation']
    
   
    train_val_df, test_df = train_test_split(pcs_df, test_size=0.2, random_state=42)
    train_df, val_df = train_test_split(train_val_df, test_size=0.15, random_state=42) # 0.15 * 0.8 = 0.12 of total

    print(f"Training samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")
    print(f"Test samples: {len(test_df)}")

    
    target_means_train = train_df[components].mean().values
    target_stds_train = train_df[components].std().values
    
    target_stds_train[target_stds_train == 0] = 1.0 

    

    
    include_velocities_feature = True 
    
    train_ds = SkeletonPCSDataset(train_df, NPZ_DIR, SEQ_LEN, include_velocities=include_velocities_feature)
    val_ds = SkeletonPCSDataset(val_df, NPZ_DIR, SEQ_LEN, include_velocities=include_velocities_feature)
    test_ds  = SkeletonPCSDataset(test_df,  NPZ_DIR, SEQ_LEN, include_velocities=include_velocities_feature)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True, drop_last=True) # num_workers can be os.cpu_count()
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

   
    try:
        sample_x_raw, _ = train_ds[0] # This will be (seq_len, features_dim)
        input_dim = sample_x_raw.shape[1]
    except Exception as e:
        print(f"Could not determine input_dim from dataset sample: {e}. Defaulting.")
        
        num_joints_coords = 17 * 3 
        input_dim = num_joints_coords * 2 if include_velocities_feature else num_joints_coords
        print(f"Defaulted input_dim to: {input_dim} (assuming 17 joints, 3 coords, velocities={include_velocities_feature})")


    print(f"Model input_dim: {input_dim}")
    
    model = LSTMRegressor(input_dim, HIDDEN_DIM, NUM_LAYERS, OUTPUT_DIM, DROPOUT).to(DEVICE)
    criterion = nn.SmoothL1Loss() # Or nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=PATIENCE//2, verbose=True)

    
    print("Starting training...")
    model, history = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, NUM_EPOCHS, DEVICE, PATIENCE, MIN_DELTA, GRADIENT_CLIP_VALUE)
    
    
    plt.figure(figsize=(10, 5))
    plt.plot(history['train_loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

    
    print("\nEvaluating on the test set...")
    metrics_df, trues_denorm, preds_denorm = evaluate_model(model, test_loader, criterion, DEVICE, target_means_train, target_stds_train, components)
    print("\nTest Set Metrics:")
    print(metrics_df)

    
    for i, comp in enumerate(components):
        plt.figure(figsize=(8, 8))
        plt.scatter(trues_denorm[:, i], preds_denorm[:, i], alpha=0.6, edgecolors='w', linewidth=0.5)
        # Add a y=x line for reference
        min_val = min(trues_denorm[:, i].min(), preds_denorm[:, i].min())
        max_val = max(trues_denorm[:, i].max(), preds_denorm[:, i].max())
        plt.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2, label='Ideal Fit (y=x)')
        plt.xlabel(f"True {comp}")
        plt.ylabel(f"Predicted {comp}")
        plt.title(f"{comp}: True vs Predicted (Test Set)\nR2: {metrics_df.loc[metrics_df['Component'] == comp, 'R2'].values[0]:.3f}")
        plt.legend()
        plt.grid(True)
        plt.show()

    print("\nScript finished.")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import spearmanr
import torch
import pandas as pd


def pearson_per_dim(y_true, y_pred):
    r_vals = []
    for i in range(y_true.shape[1]):
        r = np.corrcoef(y_true[:, i], y_pred[:, i])[0, 1]
        r_vals.append(r)
    return np.array(r_vals)


def concordance_cc(y_true, y_pred):
    cc_vals = []
    for i in range(y_true.shape[1]):
        t = y_true[:, i]
        p = y_pred[:, i]
        cov = np.mean((t - t.mean()) * (p - p.mean()))
        cc = 2 * cov / (t.var() + p.var() + (t.mean() - p.mean())**2 + 1e-8)
        cc_vals.append(cc)
    return np.array(cc_vals)


model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        x_batch = x_batch.to(device)
        preds = model(x_batch)
        y_true.append(y_batch.numpy())
        y_pred.append(preds.cpu().numpy())

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)


y_true = y_true * target_stds.values + target_means.values
y_pred = y_pred * target_stds.values + target_means.values


components = ['skating_skills', 'transitions', 'performance', 'composition', 'interpretation']

mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
r2 = r2_score(y_true, y_pred, multioutput='raw_values')
pearson = pearson_per_dim(y_true, y_pred)
spearman = np.array([spearmanr(y_true[:, i], y_pred[:, i])[0] for i in range(y_true.shape[1])])
ccc = concordance_cc(y_true, y_pred)

metrics_df = pd.DataFrame({
    'Component': components,
    'MSE': mse,
    'MAE': mae,
    'R2': r2,
    'Pearson': pearson,
    'Spearman': spearman,
    'CCC': ccc
})
print(metrics_df.round(4))


sns.set(style="whitegrid")


fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()
for i, name in enumerate(components):
    ax = axes[i]
    ax.scatter(y_true[:, i], y_pred[:, i], alpha=0.5)
    ax.plot([0, 10], [0, 10], 'r--')
    ax.set_title(f'{name} (R²={r2[i]:.2f}, r={pearson[i]:.2f})')
    ax.set_xlabel('True')
    ax.set_ylabel('Predicted')
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
fig.suptitle('True vs. Predicted Scores')
plt.tight_layout()
plt.show()


fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()
for i, name in enumerate(components):
    ax = axes[i]
    avg = (y_true[:, i] + y_pred[:, i]) / 2
    diff = y_pred[:, i] - y_true[:, i]
    mean_diff = np.mean(diff)
    std_diff = np.std(diff)
    ax.scatter(avg, diff, alpha=0.5)
    ax.axhline(mean_diff, color='gray', linestyle='--')
    ax.axhline(mean_diff + 1.96 * std_diff, color='red', linestyle='--')
    ax.axhline(mean_diff - 1.96 * std_diff, color='red', linestyle='--')
    ax.set_title(f'{name} Bland–Altman')
    ax.set_xlabel('Mean of True & Predicted')
    ax.set_ylabel('Prediction Error')
fig.suptitle('Bland–Altman Plots')
plt.tight_layout()
plt.show()


fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()
for i, name in enumerate(components):
    residuals = y_pred[:, i] - y_true[:, i]
    sns.histplot(residuals, bins=30, kde=True, ax=axes[i])
    axes[i].set_title(f'{name} Residuals')
    axes[i].set_xlabel('Prediction Error')
fig.suptitle('Residual Histograms')
plt.tight_layout()
plt.show()


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# --- Dataset ---
class SkeletonPCSDataset(Dataset):
    def __init__(self, manifest_df, npz_dir, seq_len=1000):
        self.df = manifest_df
        self.npz_dir = npz_dir
        self.seq_len = seq_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        sample_id = row['sample_id']
        npz_file = os.path.join(self.npz_dir, sample_id + '.npz')
        data = np.load(npz_file)['reconstruction']
        T, J, C = data.shape
        x = data.reshape(T, J*C).astype(np.float32)

        # Normalize input sequence (per sample)
        x = (x - np.mean(x, axis=0)) / (np.std(x, axis=0) + 1e-6)

        if T < self.seq_len:
            pad = np.zeros((self.seq_len - T, J*C), dtype=np.float32)
            x = np.vstack([x, pad])
        else:
            x = x[:self.seq_len]

        y = np.array([
            row['skating_skills'],
            row['transitions'],
            row['performance'],
            row['composition'],
            row['interpretation']
        ], dtype=np.float32)

        return torch.from_numpy(x), torch.from_numpy(y)


class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_layers=2, output_dim=5, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        outs, _ = self.lstm(x)
        final = outs[:, -1, :]
        
        return self.fc(final)


NPZ_DIR = '/kaggle/input/finefs/skeleton/skeleton'
SEQ_LEN = 500
dataset = SkeletonPCSDataset(pcs_df, NPZ_DIR, seq_len=SEQ_LEN)


train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)


sample_x, _ = dataset[0]
input_dim = sample_x.shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LSTMRegressor(input_dim).to(device)

criterion = torch.nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)


for epoch in range(10):
    model.train()
    running_loss = 0.0
    for x_batch, y_batch in train_loader:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * x_batch.size(0)

    train_loss = running_loss / len(train_loader.dataset)

    
    model.eval()
    val_preds, val_targets = [], []
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val, y_val = x_val.to(device), y_val.to(device)
            out = model(x_val)
            val_preds.append(out.cpu().numpy())
            val_targets.append(y_val.cpu().numpy())

    val_preds = np.vstack(val_preds)
    val_targets = np.vstack(val_targets)
    val_loss = mean_squared_error(val_targets, val_preds)
    r2 = r2_score(val_targets, val_preds)
    mae = mean_absolute_error(val_targets, val_preds)

    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val MSE={val_loss:.4f}, R²={r2:.4f}, MAE={mae:.4f}")


for i, component in enumerate(['skating_skills', 'transitions', 'performance', 'composition', 'interpretation']):
    plt.figure(figsize=(5, 4))
    plt.scatter(val_targets[:, i], val_preds[:, i], alpha=0.5)
    plt.xlabel('True Score')
    plt.ylabel('Predicted Score')
    plt.title(f'{component} - True vs Predicted')
    plt.plot([val_targets[:, i].min(), val_targets[:, i].max()],
             [val_targets[:, i].min(), val_targets[:, i].max()], 'r--')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


class SkeletonPCSDataset(Dataset):
    def __init__(self, manifest_df, npz_dir, seq_len=6000, mean=None, std=None):
        self.df = manifest_df.reset_index(drop=True)
        self.npz_dir = npz_dir
        self.seq_len = seq_len
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        sample_id = row['sample_id']
        data = np.load(os.path.join(self.npz_dir, sample_id + '.npz'))['reconstruction']
        T, J, C = data.shape
        x = data.reshape(T, J * C).astype(np.float32)

        if T < self.seq_len:
            pad = np.zeros((self.seq_len - T, J * C), dtype=np.float32)
            x = np.vstack([x, pad])
        else:
            x = x[:self.seq_len]

        if self.mean is not None and self.std is not None:
            x = (x - self.mean) / (self.std + 1e-6)

        y = np.array([
            row['skating_skills'],
            row['transitions'],
            row['performance'],
            row['composition'],
            row['interpretation']
        ], dtype=np.float32)

        return torch.from_numpy(x), torch.from_numpy(y)


class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, num_layers=2, output_dim=5, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        outs, _ = self.lstm(x)
        return self.fc(outs[:, -1, :])


def compute_mean_std(npz_dir, sample_ids, seq_len):
    all_data = []
    for sid in sample_ids:
        x = np.load(os.path.join(npz_dir, sid + '.npz'))['reconstruction']
        T, J, C = x.shape
        x = x.reshape(T, J*C)
        if T < seq_len:
            pad = np.zeros((seq_len - T, J*C))
            x = np.vstack([x, pad])
        else:
            x = x[:seq_len]
        all_data.append(x)
    all_data = np.stack(all_data)
    mean = np.mean(all_data, axis=(0, 1))
    std = np.std(all_data, axis=(0, 1))
    return mean.astype(np.float32), std.astype(np.float32)


SEQ_LEN = 6000
BATCH_SIZE = 16
EPOCHS = 20
NPZ_DIR = '/kaggle/input/finefs/skeleton/skeleton'

# Split dataset
train_df, test_df = train_test_split(pcs_df, test_size=0.2, random_state=42)
mean, std = compute_mean_std(NPZ_DIR, train_df['sample_id'], SEQ_LEN)

train_set = SkeletonPCSDataset(train_df, NPZ_DIR, seq_len=SEQ_LEN, mean=mean, std=std)
test_set  = SkeletonPCSDataset(test_df, NPZ_DIR, seq_len=SEQ_LEN, mean=mean, std=std)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

sample_x, _ = train_set[0]
input_dim = sample_x.shape[1]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMRegressor(input_dim=input_dim).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)


for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x_batch.size(0)

    model.eval()
    test_preds, test_targets = [], []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch = x_batch.to(device)
            preds = model(x_batch).cpu().numpy()
            test_preds.append(preds)
            test_targets.append(y_batch.numpy())

    test_preds = np.vstack(test_preds)
    test_targets = np.vstack(test_targets)

    epoch_loss = train_loss / len(train_set)
    r2 = r2_score(test_targets, test_preds)
    print(f"Epoch {epoch+1} | Train Loss: {epoch_loss:.4f} | R²: {r2:.4f}")


mae = mean_absolute_error(test_targets, test_preds)
rmse = np.sqrt(mean_squared_error(test_targets, test_preds))
r2 = r2_score(test_targets, test_preds)

print(f"\nFinal Evaluation:\nMAE: {mae:.4f} | RMSE: {rmse:.4f} | R²: {r2:.4f}")


fig, axes = plt.subplots(1, 5, figsize=(20, 4))
titles = ['Skating Skills', 'Transitions', 'Performance', 'Composition', 'Interpretation']
for i in range(5):
    axes[i].scatter(test_targets[:, i], test_preds[:, i], alpha=0.5)
    axes[i].plot([0, 10], [0, 10], 'r--')
    axes[i].set_title(titles[i])
    axes[i].set_xlabel('True')
    axes[i].set_ylabel('Predicted')
plt.tight_layout()
plt.show()